In [2]:
# %%time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from dataclasses import dataclass, field
from typing import Optional
import contextlib

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    # AutoTokenizer, 
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)


from trl import SFTTrainer

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

In [3]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype="bfloat16",
#     bnb_4bit_use_double_quant=False,
# )

# device_map = {"": 0}
base_model = "bigcode/starcoderbase-1b"
# model = "smangrul/full-finetune-starcoderbase-3b-deepspeed-colab"
model = "../personal_copilot/training/full-finetune-starcoderbase-1b-deepspeed-colab/"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model, quantization_config=None, 
    device_map=None, 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16,
)

if not hasattr(model, "hf_device_map"):
    model.cuda()

In [4]:
def get_code_completion(prefix, suffix):
    text = prompt = f"""<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>"""
    model.eval()
    outputs = model.generate(
        input_ids=tokenizer(
            text, return_tensors="pt").input_ids.cuda(), 
            max_new_tokens=512,
            temperature=0.3,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            repetition_penalty=1.2,
            #stopping_criteria=stopping_criteria
        )
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

In [5]:
prefix = """
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
"""

suffix = """
two_sum([3,4,5], 7)
"""
print(get_code_completion(prefix, suffix))

/home/nauman/miniconda3/envs/llm_workshop/lib/python3.11/site-packages/transformers/generation/utils.py:1553: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
<fim_suffix>
two_sum([3,4,5], 7)
<fim_middle>    arr = sorted()
    n = target_sum - arr[target_sum]
	if len(n) == 0:
		return arr[-1] + n], n[-1] + target_sum
# We can now use `two_sum` function to find the sum in constant-time O(N) way`.<|endoftext|>


In [6]:
prefix = """from peft import LoraConfig, TaskType

peft_config = LoraConfig("""

suffix = ")"
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from peft import LoraConfig, TaskType

peft_config = LoraConfig(<fim_suffix>)<fim_middle>
            lora_type="lora",
            lora_alpha=10.0,
            lora_dropout=0.5,
            task_type=TaskType.LORA_SEQ_PEFT,
            lora_or_id=None,
            # The name of the model checkpoint to be loaded from the model Hub (repo id or path of local file)
            lora_checkpoint="lora-weights/lora-lora-model_default_lora_state.pt" if args.task == "lora_seq" else args.lora_checkpoint
        >>> peft_config.lora_layers[i].name` for all layers in lora config and `task_type` as a string with default value 'default'
            or get_active_lora_layer(args, "lora") == TaskType.DEFAULT_LORA_LAYER_TYPE_NAME,
            lora_alpha_init=args.lora_alpha_init,
            lora_dropout_prob=args.lora_ora_dropout_prob,
            )

        return lora_config

    task_types = TaskType.get_<|endoftext|>


In [7]:
prefix = """from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, training_dataloader, scheduler = """

suffix = """"""
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, training_dataloader, scheduler = <fim_suffix><fim_middle>accelerator.prepare(model)
training_dataloader = training_dataloader["train"] if args.mixed_precision == "fp16" else training_dataloader["eval"](args=training_dataloader))


def get_lr():
    return scheduler.get_last_lr()[0]


for group in model.param_groups:
        for p in group['params']:
                print("Grads per parameter:",<|endoftext|>
